In [1]:
!pip install --upgrade google-cloud-speech

In [6]:
import io
import os
from google.cloud import speech_v1p1beta1 as speech

# --- IMPORTANT: Set up your Google Cloud credentials ---
# 1. Enable the Google Cloud Speech-to-Text API in your GCP project.
# 2. Create a Service Account and download its JSON key file.
# 3. Set the GOOGLE_APPLICATION_CREDENTIALS environment variable
#    to the path of your downloaded JSON key file.
#    Example (in your terminal or .bashrc/.zshrc):
#    export GOOGLE_APPLICATION_CREDENTIALS="/path/to/your/keyfile.json"
#    (Replace with the actual path to your service account key file)


def transcribe_audio_file(
    audio_file_path: str,
    encoding: speech.RecognitionConfig.AudioEncoding,
    sample_rate_hertz: int,
    language_code: str = "en-US",
) -> str:
    """
    Transcribes an audio file using Google Cloud Speech-to-Text API.

    Args:
        audio_file_path (str): The local path to the audio file.
        encoding (speech.RecognitionConfig.AudioEncoding): The encoding of the audio file (e.g., LINEAR16, FLAC, MP3).
        sample_rate_hertz (int): The sample rate of the audio file in Hertz (e.g., 16000, 44100).
        language_code (str): The BCP-47 language tag for the audio (e.g., "en-US", "es-ES").

    Returns:
        str: The transcribed text.
    """
    try:
        # Instantiates a client
        client = speech.SpeechClient()

        # Loads the audio file into memory
        with io.open(audio_file_path, "rb") as audio_file:
            content = audio_file.read()

        audio = speech.RecognitionAudio(content=content)

        config = speech.RecognitionConfig(
            encoding=encoding,
            sample_rate_hertz=sample_rate_hertz,
            language_code=language_code,
        )

        print(f"Sending audio file '{audio_file_path}' for transcription...")
        # Detects speech in the audio file
        response = client.recognize(config=config, audio=audio)

        transcript_parts = []
        for i, result in enumerate(response.results):
            # The first alternative is the most likely one.
            alternative = result.alternatives[0]
            transcript_parts.append(alternative.transcript)
            print(f"Transcript part {i+1}: {alternative.transcript}")


        full_transcript = " ".join(transcript_parts)
        return full_transcript

    except Exception as e:
        print(f"An error occurred: {e}")
        return f"Error during transcription: {e}"

# --- Example Usage ---
if __name__ == "__main__":
    
    # --- Configuration for your audio file ---
    # Adjust these parameters based on your actual audio file's properties.
    # Common encodings: LINEAR16 (for WAV PCM), FLAC, MP3, OGG_OPUS
    # Common sample rates: 8000, 16000, 44100
    # Use 'en-US' for US English, 'es-ES' for Spanish, 'ja-JP' for Japanese, etc.
    
#     Example 2: MP3 audio file (uncomment and modify if you have an MP3)
    audio_path_mp3 = "harvard.mp3" # Replace with your actual MP3 file path
    encoding_mp3 = speech.RecognitionConfig.AudioEncoding.MP3
    sample_rate_mp3 = 44100
    language_code_mp3 = "en-US"

    print("\n--- Transcribing MP3 Audio ---")
    transcription_mp3 = transcribe_audio_file(
        audio_path_mp3,
        encoding_mp3,
        sample_rate_mp3,
        language_code_mp3
    )
    print(f"\nFull MP3 Transcription: {transcription_mp3}")



--- Transcribing MP3 Audio ---
Sending audio file 'harvard.mp3' for transcription...
Transcript part 1: the stale smell of old beer lingers it takes heat to bring out the odor
Transcript part 2:  a cold dip restores health and zest a salt pickle tastes fine with ham tacos al pastor are my favorite a zestful food is the hot cross bun

Full MP3 Transcription: the stale smell of old beer lingers it takes heat to bring out the odor  a cold dip restores health and zest a salt pickle tastes fine with ham tacos al pastor are my favorite a zestful food is the hot cross bun
